In [1]:
import sys; sys.path.append("../")

import pandas as pd

from lib.match import (
    ColumnsIndex, JaroWinklerSimilarity, StringSimilarity, DateSimilarity, ThresholdMatcher
)
from lib.date import combine_date_columns
from clean.new_orleans_harbor_pd_pprr import clean_personnel
from match.new_orleans_harbor_pd import prepare_post_data
from lib.columns import rearrange_personnel_columns, rearrange_personnel_history_columns

In [2]:
df = clean_personnel()
df

/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
../lib/clean.py:166: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1842: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
../lib/clean.py:156: FutureWarnin

,first_name,last_name,rank_desc,hourly_salary,middle_initial,term_year,term_month,term_day,hire_year,hire_month,hire_day,pay_effective_year,pay_effective_month,pay_effective_day,uid,agency,data_production_year
0,casey,adams,police sergeant--port,17.00,m,2010,2,28,1980,1,31,1997,4,14,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
1,casey,adams,police sergeant--port,17.68,m,2010,2,28,1980,1,31,2003,12,10,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
2,casey,adams,police sergeant--port,18.39,m,2010,2,28,1980,1,31,2004,8,10,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
3,casey,adams,police sergeant--port,19.13,m,2010,2,28,1980,1,31,2005,8,10,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
4,casey,adams,police sergeant-a,19.13,m,2010,2,28,1980,1,31,2005,8,29,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,brianne,waters,police officer 3-a,21.60,m,2020,5,11,2001,3,5,2015,10,1,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020
911,brianne,waters,police sergeant-a,24.34,m,2020,5,11,2001,3,5,2018,5,14,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020
912,brianne,waters,police sergeant-a,25.07,m,2020,5,11,2001,3,5,2018,7,15,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020
913,brianne,waters,police sergeant-a,25.07,m,2020,5,11,2001,3,5,2018,7,15,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020


In [3]:
post = prepare_post_data()
post

,agency,last_name,first_name,employment_status,level_1_cert_date,last_pc_12_qualification_date,data_production_year,hire_year,hire_month,hire_day,uid
uid,,,,,,,,,,,
1b0db8db512b965ac050e1f80c3b16cd,new orleans harbor pd,trigueros,jennifer,full-time,2013-05-01,2020-10-06,2020,2017.0,12.0,4.0,1b0db8db512b965ac050e1f80c3b16cd
df40c69b2f569301db5ba096bd1066ba,new orleans harbor pd,savoie,mickael,full-time,NaN,2020-06-04,2020,2007.0,4.0,23.0,df40c69b2f569301db5ba096bd1066ba
bceaf0862630ae98d4a17d8483f7a654,new orleans harbor pd,clark,chris,full-time,2012-01-18,2020-06-04,2020,2012.0,6.0,4.0,bceaf0862630ae98d4a17d8483f7a654
605529f07facfb432ee3ae019e908c7f,new orleans harbor pd,smith,christopher,full-time,2012-10-04,2020-06-04,2020,2012.0,5.0,24.0,605529f07facfb432ee3ae019e908c7f
71db98a7bcb893f3f7371accfac32475,new orleans harbor pd,cook,joseph,full-time,2014-05-21,2020-06-04,2020,2014.0,3.0,7.0,71db98a7bcb893f3f7371accfac32475
3c3c70dcb8c0841fc453cae8b2ede855,new orleans harbor pd,favaloro,john,full-time,2018-03-21,2020-06-04,2020,2017.0,11.0,6.0,3c3c70dcb8c0841fc453cae8b2ede855
f1e1a63440a61a93b9b41b89a04648ef,new orleans harbor pd,causey,travis,full-time,2017-04-27,2020-06-04,2020,2019.0,2.0,25.0,f1e1a63440a61a93b9b41b89a04648ef
ee9bfdc0586b4a2641bf1cfd36ce4152,new orleans harbor pd,gordon,reginald,full-time,NaN,2020-06-02,2020,2017.0,3.0,1.0,ee9bfdc0586b4a2641bf1cfd36ce4152
aa514b50f5dbbe40dafe5669f564242c,new orleans harbor pd,sylvas,shaun,full-time,2018-03-21,2020-06-02,2020,2017.0,9.0,5.0,aa514b50f5dbbe40dafe5669f564242c


In [4]:
dfa = df[['uid', 'first_name', 'last_name']]
dfa.loc[:, "hire_date"] = combine_date_columns(
    df, "hire_year", "hire_month", "hire_day")
dfa.loc[:, 'fc'] = dfa.first_name.map(lambda x: x[:1])
dfa = dfa.drop_duplicates(subset=['uid']).set_index('uid')

dfb = post[['uid', 'first_name', 'last_name']]
dfb.loc[:, "hire_date"] = combine_date_columns(
    post, "hire_year", "hire_month", "hire_day")
dfb.loc[:, 'fc'] = dfb.first_name.map(lambda x: x[:1])
dfb = dfb.drop_duplicates(subset=['uid']).set_index('uid')

matcher = ThresholdMatcher(dfa, dfb, ColumnsIndex(["fc"]), {
    "last_name": JaroWinklerSimilarity(),
    "first_name": JaroWinklerSimilarity()
})
matcher.get_sample_pairs(sample_counts=20, step=0.025)

/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1675: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


first_name  \
score_range pair_idx sim_score row_key                                         
1.00-0.97   0        1.000000  baacff310946f3b7d9f7f6c9f8250341        logan   
                               0267a2624028c3df6f4c35d75ff7d52a        logan   
            1        1.000000  33a79dbef4ad1582d9db94531fa2640c         drew   
                               6f810c02bb018c9a64652aacf1109895         drew   
            2        1.000000  e37d798e6c31d343ef2fefa1d4882914       darryl   
                               77f8a0e5d954d3d7bd62ad38abacc831       darryl   
            3        1.000000  d102f30880453bdfdcb68b4551177836     reginald   
                               ee9bfdc0586b4a2641bf1cfd36ce4152     reginald   
            4        1.000000  af467efe8ab36656df9bd59b40736c48       travis   
                               f1e1a63440a61a93b9b41b89a04648ef       travis   
            5        1.000000  93c6c3ae4c473c3af3692351637f551a     jennifer   
                               1b0db8db512b965ac050e1f80c3b16cd     jennifer   
            6        1.000000  a542ea4947e2e2e0e13269f71e922b0a       johnny   
                               ab335d5cd87cd9fa6b27bbb8549508a4       johnny   
            7        1.000000  3961d9f48d0ba23e5cdc4cb72e344ab0       joseph   
                               71db98a7bcb893f3f7371accfac32475       joseph   
            8        1.000000  2c881fed98ae1d2d98c5452e3a34b5db        byron   
                               2c77e722124051bcebf7ce7ed6583f28        byron   
            9        1.000000  a96cc30d728230eb421183c9f85c1b79  christopher   
                               605529f07facfb432ee3ae019e908c7f  christopher   
            10       1.000000  35af1847b6183ab2fc622c09fbf46428        shaun   
                               aa514b50f5dbbe40dafe5669f564242c        shaun   
            11       1.000000  15bc9c7d64331dcde0f899a6c3d724e1      mickael   
                               df40c69b2f569301db5ba096bd1066ba      mickael   
            12       1.000000  0adb29b27a30568c9dda5eee62b4de57      maurice   
                               872f4fd2a63af742ec413d1eb904fb91      maurice   
            13       0.990951  459afaee00f60a7a703d63576e351716         john   
                               3c3c70dcb8c0841fc453cae8b2ede855         john   
0.97-0.95   0        0.969254  5cf305e49d5866c801f60045568ec78b        chris   
                               bceaf0862630ae98d4a17d8483f7a654        chris   

                                                                   last_name  \
score_range pair_idx sim_score row_key                                         
1.00-0.97   0        1.000000  baacff310946f3b7d9f7f6c9f8250341      tillery   
                               0267a2624028c3df6f4c35d75ff7d52a      tillery   
            1        1.000000  33a79dbef4ad1582d9db94531fa2640c     mercadel   
                               6f810c02bb018c9a64652aacf1109895     mercadel   
            2        1.000000  e37d798e6c31d343ef2fefa1d4882914       barnes   
                               77f8a0e5d954d3d7bd62ad38abacc831       barnes   
            3        1.000000  d102f30880453bdfdcb68b4551177836       gordon   
                               ee9bfdc0586b4a2641bf1cfd36ce4152       gordon   
            4        1.000000  af467efe8ab36656df9bd59b40736c48       causey   
                               f1e1a63440a61a93b9b41b89a04648ef       causey   
            5        1.000000  93c6c3ae4c473c3af3692351637f551a    trigueros   
                               1b0db8db512b965ac050e1f80c3b16cd    trigueros   
            6        1.000000  a542ea4947e2e2e0e13269f71e922b0a       helire   
                               ab335d5cd87cd9fa6b27bbb8549508a4       helire   
            7        1.000000  3961d9f48d0ba23e5cdc4cb72e344ab0         cook   
                               71db98a7bcb893f3f7371accfac32475         cook   
            8        1.000000  2c881fed98ae1d2d98c5452e3a34b5db  